# A - 3 - Full Texts Acquisition

## Description
**Process aim:**
This process aims at adding a field containing the full text of the resources to the metadata dataframe.

**Input:** A csv files containing metadata including at least one columns with URLs

**Sub-processes**:
1. Import metadata
2. Get and save PDF files
3. Extract full texts from PDF files
4. Add full text to dataset and save

**Output:** a CSV file

## 1. Import metadata

In [1]:
import pandas as pd
import requests
import textract

In [2]:
md_dataset = (pd.read_csv('data/acquisition/dataset_from_MARC.csv')
             .set_index('record_id'))

In [3]:
md_dataset

,body,corporates,geographic_terms,session,symbol,title,topics,url-en,url-es,url-fr,url-ru,url-ar,url-zh
record_id,,,,,,,,,,,,,
703243,A/,NaN,NaN,65,A/65/PV.71,"General Assembly official records, 65th sessio...","['SOCIAL DEVELOPMENT', 'AGEING PERSONS', 'DEMO...",http://digitallibrary.un.org/record/703243/fil...,http://digitallibrary.un.org/record/703243/fil...,http://digitallibrary.un.org/record/703243/fil...,http://digitallibrary.un.org/record/703243/fil...,http://digitallibrary.un.org/record/703243/fil...,http://digitallibrary.un.org/record/703243/fil...
703245,A/,['UN. Peacebuilding Commission. Organizational...,NaN,65,A/65/PV.72,"General Assembly official records, 65th sessio...","['POPULATION PROGRAMMES', 'CHEMICAL WEAPONS', ...",http://digitallibrary.un.org/record/703245/fil...,http://digitallibrary.un.org/record/703245/fil...,http://digitallibrary.un.org/record/703245/fil...,http://digitallibrary.un.org/record/703245/fil...,http://digitallibrary.un.org/record/703245/fil...,http://digitallibrary.un.org/record/703245/fil...
701732,A/,['UN. General Assembly (66th sess. : 2011-2012...,NaN,65,A/65/PV.57,"General Assembly official records, 65th sessio...","['STATE RESPONSIBILITY', 'STANDARDS OF CONDUCT...",http://digitallibrary.un.org/record/701732/fil...,http://digitallibrary.un.org/record/701732/fil...,http://digitallibrary.un.org/record/701732/fil...,http://digitallibrary.un.org/record/701732/fil...,http://digitallibrary.un.org/record/701732/fil...,http://digitallibrary.un.org/record/701732/fil...
730980,A/,"['UN. Board of Auditors', 'UN System', 'UN. Of...",MYANMAR,66,A/66/PV.93,"General Assembly official records, 66th sessio...","['PERSONS WITH DISABILITIES', 'HUMAN RIGHTS', ...",http://digitallibrary.un.org/record/730980/fil...,http://digitallibrary.un.org/record/730980/fil...,http://digitallibrary.un.org/record/730980/fil...,http://digitallibrary.un.org/record/730980/fil...,http://digitallibrary.un.org/record/730980/fil...,http://digitallibrary.un.org/record/730980/fil...
730711,A/,"['Unesco', 'UN Trust Fund for Partnerships - P...",NaN,66,A/66/PV.83,"General Assembly official records, 66th sessio...","['HEALTH POLICY', 'FOREIGN POLICY', 'MEMORIALS...",http://digitallibrary.un.org/record/730711/fil...,http://digitallibrary.un.org/record/730711/fil...,http://digitallibrary.un.org/record/730711/fil...,http://digitallibrary.un.org/record/730711/fil...,http://digitallibrary.un.org/record/730711/fil...,http://digitallibrary.un.org/record/730711/fil...
730697,A/,"['UNRWA', 'UN. Special Committee to Investigat...",NaN,66,A/66/PV.81,"General Assembly official records, 66th sessio...","['MINE CLEARANCE', 'OUTER SPACE', 'PALESTINE R...",http://digitallibrary.un.org/record/730697/fil...,http://digitallibrary.un.org/record/730697/fil...,http://digitallibrary.un.org/record/730697/fil...,http://digitallibrary.un.org/record/730697/fil...,http://digitallibrary.un.org/record/730697/fil...,http://digitallibrary.un.org/record/730697/fil...
710829,A/,UN Interim Force in Lebanon,NaN,65,A/65/PV.106,"General Assembly official records, 65th sessio...","['SPECIAL MISSIONS', 'PEACEKEEPING OPERATIONS'...",http://digitallibrary.un.org/record/710829/fil...,http://digitallibrary.un.org/record/710829/fil...,http://digitallibrary.un.org/record/710829/fil...,http://digitallibrary.un.org/record/710829/fil...,http://digitallibrary.un.org/record/710829/fil...,http://digitallibrary.un.org/record/710829/fil...
702840,A/,International Commission against Impunity in G...,GUATEMALA,65,A/65/PV.69,"General Assembly official records, 65th sessio...","['HUMAN RIGHTS', 'INFORMATION TECHNOLOGY', 'CO...",http://digitallibrary.un.org/record/702840/fil...,http://digitallibrary.un.org/record/702840/fil...,http://digitallibrary.un.org/record/702840/fil...,http://digitallibrary.un.org/record/702840/fil...,http://digitallibrary.un.org/record/702840/fil...,http://digitallibrary.un.org/record/702840/fil...
702722,A/,['UN. General Assembly (65th sess. : 2010-2011...,MYANMAR,

## 2. Get and save PDF files
We need the full text of the resources described in the MARC XML that will be used later to infer some metadata. In this case we will focus on English texts only.

For this step start by creating a list containing for each record that has an English url, the record id, and the url. We then use the function save_files() to get the files using the url and save them in the the folder data/acquisition.

In [4]:
def save_files(files_list, save_path, file_extension):
    '''
    Takes a list of of lists (record_id and url), the path of the location where the files
    will be saved, and the extension of the file type. Get the files through htpp requests
    and save them. Returns a list of record_id, corresponding to files that could not be 
    saved.
    '''
    errors = []
    for item in files_list:
        save_as = save_path + str(item[0]) + file_extension
        file_url = item[1]
        response = requests.get(file_url)
        if response.status_code == requests.codes.ok:
            with open(save_as, 'wb') as f:
                f.write(response.content)
        else:
            not_saved.append(item)
    return errors

In [5]:
# From md_dataset, we create a list of lists with 2 values. The record_id and the url of the English file
en_list = (md_dataset.reset_index()[['record_id', 'url-en']]
           .dropna() # filter out if non values
           .values.tolist())
# Get the files and same them in pdf
save_files(en_list, 'data/acquisition/', '.pdf')

[]

## 3. Extract full text from PDF files
Using the same files list we then use the convert_to_pdf_function, to get the content of the PDFs, convert it to a string of text, and store this as a third column in the initial list. Note that if a page cannot be processed, then it will be skipped altogether.

In [6]:
def convert_pdf_to_text(files_list, path):
    '''
    Takes a list of list, and a path to files in pdf. Read each files and convert to text.
    Append the resulting texts to the initial list and return the list.
    '''
    new_list = []
    for file in files_list:
        file_path = path + str(file[0]) + '.pdf'
        full_text = textract.process(file_path)
        full_text= full_text.decode()
        file.append(full_text)    
        new_list.append(file)
    return new_list

In [7]:
# Add a column with the full text of the pdf
en_list = convert_pdf_to_text(en_list,'data/acquisition/')

## 4. Add full texts to the metadata and save the output
To finish, we create a new dataframe with only the record_id and the full text. As they have the same index, the record-id, we can join it to the metadata dataframe and easily save the result as a CSV.

In [8]:
# Create a new dataframe withg the record_id and the full text
full_text = (pd.DataFrame(en_list, columns=['record_id','url','text'])
             .drop('url', axis=1)
             .set_index('record_id')
            )
# Join the result to the metadta dataset
md_dataset = md_dataset.join(full_text)
# Check the result
md_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 703243 to 702423
Data columns (total 14 columns):
body                200 non-null object
corporates          162 non-null object
geographic_terms    90 non-null object
session             199 non-null object
symbol              200 non-null object
title               200 non-null object
topics              193 non-null object
url-en              198 non-null object
url-es              195 non-null object
url-fr              198 non-null object
url-ru              197 non-null object
url-ar              194 non-null object
url-zh              196 non-null object
text                198 non-null object
dtypes: object(14)
memory usage: 33.4+ KB


In [9]:
# Save the content of the dataset in data/pre-processing/
md_dataset.to_csv('data/pre-processing/dataset_from_MARC_text.csv')